In [1]:
# Download directly from Mendeley (version 3, ~267MB)
!wget -q https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/n3gtgm9jxj-3.zip -O waste_dataset.zip

# Unzip
!unzip -q waste_dataset.zip -d /content/waste_data


In [2]:
#Check the Real Directory Structure
import os

import os

# List top-level directory
print("Top-level:", os.listdir("/content/waste_data"))

# List subfolders inside waste_data
for root, dirs, files in os.walk("/content/waste_data"):
    print("📁", root)
    for d in dirs:
        print("   └──", d)
    break



Top-level: ['Waste Classification Dataset']
📁 /content/waste_data
   └── Waste Classification Dataset


In [3]:
#update path
base_dir = '/content/waste_data/Waste Classification Dataset'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'test')


In [4]:
import os

for root, dirs, files in os.walk("/content/waste_data"):
    print(f"\n📁 {root}")
    for d in dirs:
        print(f" └── {d}")




📁 /content/waste_data
 └── Waste Classification Dataset

📁 /content/waste_data/Waste Classification Dataset
 └── waste_dataset

📁 /content/waste_data/Waste Classification Dataset/waste_dataset
 └── organic
 └── recyclable

📁 /content/waste_data/Waste Classification Dataset/waste_dataset/organic

📁 /content/waste_data/Waste Classification Dataset/waste_dataset/recyclable


In [5]:
#Define correct path
import os

base_dir = '/content/waste_data/Waste Classification Dataset/waste_dataset'
print("Classes:", os.listdir(base_dir))  # Should print ['recyclable', 'organic']


Classes: ['organic', 'waste_dataset_CNN.ipynb', 'waste_dataset_augmentation.ipynb', 'waste_dataset_README.txt', 'recyclable']


In [6]:
#Create a train/test split (using split_folders)
!pip install split-folders

import splitfolders

# Create split dataset folders
splitfolders.ratio(base_dir, output="/content/waste_split", seed=42, ratio=(.8, .2))


Copying files: 24705 files [00:04, 5058.78 files/s]


In [7]:
#Load images for training
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (160, 160)
batch_size = 32

train_dir = "/content/waste_split/train"
val_dir = "/content/waste_split/val"

datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=img_size,
                                        batch_size=batch_size, class_mode='categorical')

val_gen = datagen.flow_from_directory(val_dir, target_size=img_size,
                                      batch_size=batch_size, class_mode='categorical')


Found 19764 images belonging to 2 classes.
Found 4941 images belonging to 2 classes.


In [10]:
##Load and train the MobileNetV2 Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import tensorflow as tf

# Define paths
train_dir = "/content/waste_split/train"
val_dir = "/content/waste_split/val"

# Set image size and batch size
img_size = (160, 160)
batch_size = 32

# Load and normalize images
datagen = ImageDataGenerator(rescale=1./255)

train_gen = datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
val_gen   = datagen.flow_from_directory(val_dir,   target_size=img_size, batch_size=batch_size, class_mode='categorical')

# Build lightweight MobileNetV2 model
base_model = MobileNetV2(input_shape=(160,160,3), include_top=False, weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_gen.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_gen, validation_data=val_gen, epochs=5)

# Save model
model.save("waste_model.keras")

Found 19764 images belonging to 2 classes.
Found 4941 images belonging to 2 classes.
Epoch 1/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 0s 607ms/step - accuracy: 0.9061 - loss: 0.2304

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


618/618 ━━━━━━━━━━━━━━━━━━━━ 525s 839ms/step - accuracy: 0.9062 - loss: 0.2304 - val_accuracy: 0.9417 - val_loss: 0.1591
Epoch 2/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 418s 676ms/step - accuracy: 0.9549 - loss: 0.1179 - val_accuracy: 0.9470 - val_loss: 0.1514
Epoch 3/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 425s 688ms/step - accuracy: 0.9686 - loss: 0.0824 - val_accuracy: 0.9431 - val_loss: 0.1791
Epoch 4/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 458s 741ms/step - accuracy: 0.9848 - loss: 0.0473 - val_accuracy: 0.9492 - val_loss: 0.1692
Epoch 5/5
618/618 ━━━━━━━━━━━━━━━━━━━━ 435s 704ms/step - accuracy: 0.9910 - loss: 0.0275 - val_accuracy: 0.9482 - val_loss: 0.1810


In [11]:
#Converting to TensorFlow Lite
import tensorflow as tf

# Load the saved Keras model
model = tf.keras.models.load_model("waste_model.keras")

# Convert the Keras model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("waste_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model converted to TFLite and saved as waste_model.tflite")

Saved artifact at '/tmp/tmpssxlzp4u'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  140119596510544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596511888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596509392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596509008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596510736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596510352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596510160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596512080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140119596504016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1401195965145

In [12]:
#Inference Test
from PIL import Image
import numpy as np

interpreter = tf.lite.Interpreter(model_path="waste_model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

# Load a sample image from validation set
sample_path = val_dir + '/organic/' + os.listdir(val_dir + '/organic')[0]
img = Image.open(sample_path).resize((160,160))
x = np.expand_dims(np.array(img)/255.0, axis=0).astype(np.float32)

interpreter.set_tensor(input_index, x)
interpreter.invoke()

pred = interpreter.get_tensor(output_index)
print("Class indices:", train_gen.class_indices)
print("Predicted class:", np.argmax(pred))


Class indices: {'organic': 0, 'recyclable': 1}
Predicted class: 0


##  Edge AI Benefits (Write-up)

Edge AI enables **real-time classification** directly on embedded devices like Raspberry Pi. Key benefits:

-  **Low Latency**: No internet or cloud roundtrips needed.
-  **Better Privacy**: No images are sent to cloud servers.
-  **Efficient**: Great for smart bins, sorting robots, or mobile apps.

In [14]:
#Deployment
from google.colab import files
files.download("waste_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>